In [407]:
from collections import defaultdict

In [408]:
possibly_something = {}

In [423]:
ingredientalergens = defaultdict(set)
alergeningredients = defaultdict(set)

allingredients = set()
allalergens = set()

recipes = []

with open('input.txt') as f:
    for line in f:
        line = line.strip()
        ingredientlist, containslist = line.split('(contains ')
        ingredients = ingredientlist.strip().split()
        alergens = containslist.rstrip(')').split(', ')

        
        recipes.append((set(ingredients), set(alergens)))
        
        allingredients.update(ingredients)
        allalergens.update(alergens)
        
        for alergen in alergens:
            alergen_ingredient_lists[alergen]

            for ingredient in ingredients:
                ingredientalergens[ingredient].add(alergen)
                alergeningredients[alergen].add(ingredient)

In [411]:
recipes_by_alergens = list(sorted(recipes, key=lambda x: len(x[1])))

In [437]:
shortest_ingredients_by_alergens = {}
for ingredients, alergens in recipes:
    for alergen in alergens:
        if alergen not in shortest_ingredients_by_alergens:
            shortest_ingredients_by_alergens[alergen] = ingredients.copy()
        else:
            shortest_ingredients_by_alergens[alergen] &= ingredients


In [438]:
shortest_ingredients_by_alergens

{'shellfish': {'bfgcms', 'fkcmf', 'ktpbgdn'},
 'dairy': {'fkcmf', 'ktpbgdn'},
 'eggs': {'hdqkqhh', 'ktpbgdn', 'pnpfjb'},
 'peanuts': {'bfgcms', 'fkcmf', 'ndfb', 'pnpfjb', 'rdhljms'},
 'wheat': {'fkcmf', 'hdqkqhh', 'ndfb'},
 'sesame': {'xzfj'},
 'fish': {'ndfb', 'xzfj'},
 'soy': {'fkcmf', 'xzfj'}}

In [439]:
def assign_alergens(assigned, unassigned_ingredients, unassigned_alergens, level=0):
    if len(unassigned_alergens) == 0:

        for ingredients, alergens in recipes:
            assigned_alergens = {assigned[ingredient] for ingredient in ingredients if ingredient in assigned}
            if len(alergens - assigned_alergens) > 0:
                return False
        else:
            return assigned
                
    for alergen in unassigned_alergens:
        ingredients = shortest_ingredients_by_alergens[alergen]
        for ingredient in unassigned_ingredients.intersection(ingredients):
            new_assigned = assigned.copy()
            new_assigned[ingredient] = alergen

            result = assign_alergens(
                new_assigned, 
                unassigned_ingredients - {ingredient},
                unassigned_alergens - {alergen},
                level=level+1
            )
            if result is not False:
                return result
        
    return False

In [440]:
unassigned_ingredients = allingredients
unassigned_alergens = allalergens

In [441]:
assignment = assign_alergens(assigned, unassigned_ingredients, unassigned_alergens)

In [442]:
assignment

{'hdqkqhh': 'wheat',
 'ktpbgdn': 'dairy',
 'fkcmf': 'soy',
 'ndfb': 'fish',
 'bfgcms': 'shellfish',
 'rdhljms': 'peanuts',
 'xzfj': 'sesame',
 'pnpfjb': 'eggs'}

Correct assignment:

```
{'mxmxvkd': 'dairy', 'sqjhc': 'fish', 'fvjkl': 'soy'}
```

In [443]:
unassigned = allingredients - set(assignment)

In [444]:
count = 0
for ingredient in unassigned:
    for ingredients, alergens in recipes:
        if ingredient in ingredients:
            count += 1

In [445]:
count

2380

In [446]:
','.join(x[0] for x in sorted(assignment.items(), key=lambda x: x[1]))

'ktpbgdn,pnpfjb,ndfb,rdhljms,xzfj,bfgcms,fkcmf,hdqkqhh'